<a href="https://colab.research.google.com/github/esradag/airbnb-ai-search-llamaindex-mongodb-gradio/blob/main/airbnb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -qU llama-index
!pip install -qU llama-index-vector-stores-mongodb
!pip install -qU llama-index-llms-openai
!pip install -qU llama-index-embeddings-openai
!pip install -qU pymongo pandas datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curre

In [3]:
import os
import getpass
from pymongo import MongoClient

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key:")
MONGODB_URI = getpass.getpass("Enter your MongoDB URI: ")

mongodb_client = MongoClient(
    MONGODB_URI, appname="devrel.content.airbnb_agent_mongodb_llamaindex"
)


Enter OpenAI API Key:··········
Enter your MongoDB URI: ··········


In [4]:
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
    dimensions=256,
    embed_batch_size=10,
    openai_api_key=os.environ["OPENAI_API_KEY"],
)
llm = OpenAI(model="gpt-4o", temperature=0)


In [5]:
import pandas as pd
from datasets import load_dataset

data = load_dataset("MongoDB/airbnb_embeddings", split="train", streaming=True)
data = data.take(200)
data_df = pd.DataFrame(data)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.84k [00:00<?, ?B/s]

In [6]:
from llama_index.core import Document

docs = data_df.to_dict(orient="records")

llama_documents = []
fields_to_include = [
    "amenities",
    "address",
    "availability",
    "review_scores",
    "listing_url",
    "price",
    "room_type",
    "accommodates",
    "cancellation_policy"
]

for doc in docs:
    metadata = {key: doc.get(key, None) for key in fields_to_include}
    llama_doc = Document(text=doc["description"], metadata=metadata)
    llama_documents.append(llama_doc)


In [7]:
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch
from pymongo.errors import OperationFailure

DB_NAME = "airbnb"
COLLECTION_NAME = "listings_reviews"
VS_INDEX_NAME = "vector_index"
FTS_INDEX_NAME = "fts_index"
collection = mongodb_client[DB_NAME][COLLECTION_NAME]

vector_store = MongoDBAtlasVectorSearch(
    mongodb_client,
    db_name=DB_NAME,
    collection_name=COLLECTION_NAME,
    vector_index_name=VS_INDEX_NAME,
    fulltext_index_name=FTS_INDEX_NAME,
    embedding_key="embedding",
    text_key="text",
)
vector_store_context = StorageContext.from_defaults(vector_store=vector_store)
vector_store_index = VectorStoreIndex.from_documents(
    llama_documents, storage_context=vector_store_context, show_progress=True
)


Parsing nodes:   0%|          | 0/200 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/200 [00:00<?, ?it/s]

In [8]:
from typing import List, Optional
from llama_index.core.vector_stores import (
    FilterCondition,
    FilterOperator,
    MetadataFilter,
    MetadataFilters,
)

def get_airbnb_listings(
    query: str,
    amenities: List[str] = [],
    min_price: Optional[int] = None,
    max_price: Optional[int] = None,
    room_type: Optional[str] = None,
    min_guests: Optional[int] = None,
    cancellation_policy: Optional[str] = None
) -> List[str]:

    filters = [
        MetadataFilter(
            key="metadata.review_scores.review_scores_rating",
            value=80,
            operator=FilterOperator.GTE,
        )
    ]

    if amenities:
        amenities_filters = [
            MetadataFilter(
                key="metadata.amenities", value=amenity, operator=FilterOperator.EQ
            )
            for amenity in amenities
        ]
        filters.extend(amenities_filters)

    if min_price is not None:
        filters.append(
            MetadataFilter(
                key="metadata.price",
                value=min_price,
                operator=FilterOperator.GTE,
            )
        )
    if max_price is not None:
        filters.append(
            MetadataFilter(
                key="metadata.price",
                value=max_price,
                operator=FilterOperator.LTE,
            )
        )

    if room_type:
        filters.append(
            MetadataFilter(
                key="metadata.room_type",
                value=room_type,
                operator=FilterOperator.EQ,
            )
        )

    if min_guests is not None:
        filters.append(
            MetadataFilter(
                key="metadata.accommodates",
                value=min_guests,
                operator=FilterOperator.GTE,
            )
        )

    if cancellation_policy:
        filters.append(
            MetadataFilter(
                key="metadata.cancellation_policy",
                value=cancellation_policy,
                operator=FilterOperator.EQ,
            )
        )

    filters = MetadataFilters(
        filters=filters,
        condition=FilterCondition.AND,
    )

    query_engine = vector_store_index.as_query_engine(
        similarity_top_k=5, vector_store_query_mode="hybrid", alpha=0.7, filters=filters
    )
    response = query_engine.query(query)
    listings = [node.metadata["listing_url"] for node in response.source_nodes]

    return listings


In [11]:
from llama_index.core.agent import AgentRunner, FunctionCallingAgentWorker
from llama_index.core.tools import FunctionTool

query_tool = FunctionTool.from_defaults(name="get_airbnb_listings", fn=get_airbnb_listings)

agent_worker = FunctionCallingAgentWorker.from_tools([query_tool], llm=llm, verbose=True)
agent = AgentRunner(agent_worker)

response = agent.query("Give me listings in Porto with a Waterfront, a flexible cancellation policy, and a review score of at least 90.")
print(response)


Added user message to memory: Give me listings in Porto with a Waterfront, a flexible cancellation policy, and a review score of at least 90.
=== Calling Function ===
Calling function: get_airbnb_listings with args: {"query": "Porto", "amenities": ["Waterfront"], "cancellation_policy": "flexible"}
=== Function Output ===
Encountered error: PlanExecutor error during aggregation :: caused by :: Path 'metadata.cancellation_policy' needs to be indexed as token, full error: {'ok': 0.0, 'errmsg': "PlanExecutor error during aggregation :: caused by :: Path 'metadata.cancellation_policy' needs to be indexed as token", 'code': 8, 'codeName': 'UnknownError', '$clusterTime': {'clusterTime': Timestamp(1739825445, 71), 'signature': {'hash': b'F\xb6"<i9qu\x07_j8R\xa8F\x80\xec\x90\rB', 'keyId': 7441629223753613375}}, 'operationTime': Timestamp(1739825445, 71)}
=== LLM Response ===
It seems there was an error while trying to retrieve listings with a flexible cancellation policy. Let me try fetching li

In [13]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [22]:
import gradio as gr
import requests
from bs4 import BeautifulSoup

# Function to extract metadata (title, description, image) from an Airbnb listing page
def get_airbnb_details(listing_url):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(listing_url, headers=headers, timeout=5)
        soup = BeautifulSoup(response.text, "html.parser")

        # Extract metadata
        title = soup.find("meta", property="og:title")
        description = soup.find("meta", property="og:description")
        image = soup.find("meta", property="og:image")

        # Ensure data exists before returning
        if title and title["content"] and description and description["content"]:
            return {
                "title": title["content"],
                "description": description["content"],
                "image_url": image["content"] if image else "https://via.placeholder.com/150"
            }

    except Exception as e:
        print(f"Error fetching details for {listing_url}: {e}")

    return None  # Return None if data is missing

# Function to format listings while ensuring uniqueness & hiding empty results
def format_listings(query):
    listings = get_airbnb_listings(query=query)

    if not listings:
        return "No listings found."

    unique_listings = list(set(listings))  # Remove duplicates
    html_output = "<div style='display: flex; flex-direction: column; gap: 15px;'>"

    for listing in unique_listings:
        details = get_airbnb_details(listing)

        if details:  # Only add listings that have valid details
            html_output += f"""
            <div style='display: flex; align-items: center; border: 1px solid #ddd; padding: 10px; border-radius: 10px; width: 90%;'>
                <img src="{details['image_url']}" width="120" height="100" style='border-radius: 10px; margin-right: 15px;'>
                <div>
                    <p style='font-size: 16px; font-weight: bold; margin: 0;'>{details['title']}</p>
                    <p style='font-size: 14px; margin: 5px 0; color: gray;'>{details['description']}</p>
                    <a href="{listing}" target="_blank" style="color: #007bff; font-size: 16px; text-decoration: none;">🔗 View Listing</a>
                </div>
            </div>
            """

    html_output += "</div>"
    return html_output if html_output != "<div style='display: flex; flex-direction: column; gap: 15px;'></div>" else "No valid listings found."

# Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("<h1 style='text-align: center;'>🏡 AI-Powered Airbnb Search</h1>")

    with gr.Row():
        search_query = gr.Textbox(label="Search Query", value="Give me listings in Porto with a Waterfront")

    search_btn = gr.Button("🔍 Search")

    listings_output = gr.HTML()

    search_btn.click(
        fn=format_listings,
        inputs=[search_query],
        outputs=[listings_output]
    )

# Launch Gradio App
demo.launch()




Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://391b3bf0a6b30ef054.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
